In [1]:
import pydantic
print(pydantic.__version__)  # should be 2.12.x

2.12.3


In [ ]:
!pip install chromadb sentence-transformers langchain langchain-core langchain-community openai tiktoken

In [ ]:
!pip install -U langchain-huggingface

In [ ]:

!pip install langchain langchain-community langchain-core langchain-openai

In [1]:
import langchain
print(langchain.__version__)

1.0.1


In [2]:
!pip list | grep langchain

langchain                                1.0.1
langchain-chroma                         1.0.0
langchain-classic                        1.0.0
langchain-community                      0.4
langchain-core                           1.0.0
langchain-huggingface                    1.0.0
langchain-openai                         1.0.0
langchain-text-splitters                 1.0.0
langchainplus-sdk                        0.0.20


In [3]:


from langchain_core.documents import Document
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
#from langchain.chains import ChatVectorDBChain
from langchain_community.chat_models import ChatOpenAI  # or from langchain_openai import ChatOpenAI

In [8]:
import os
from typing import List
from nltk.tokenize import sent_tokenize

import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/raluca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip show langchain

In [ ]:
!pip install openai

In [ ]:
!pip install -U ipywidgets

In [ ]:

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    # model_kwargs={"timeout": 60}  # increase timeout to 60 seconds
)

In [10]:

import os
import json
from datetime import datetime


In [16]:
from langchain_openai import ChatOpenAI

In [51]:

# FOR THE POEM GENERATION
llm = ChatOpenAI(temperature=0.9, model_name="gpt-4.1", openai_api_key = "YOUR_KEY")

In [45]:

# FOR THE RAG
llm_1 = ChatOpenAI(temperature=0, model_name="gpt-4.1", openai_api_key = "YOUR_KEY")

In [ ]:

# --- Configuration ---
POEMS_FOLDER = "parallel_universes_dna_corpus"
CHROMA_DB_DIR = "parallel_chroma_db"
STATE_FILE = "parallel_multiverse_state.json" # EVOLUTION OF PROMPT POEM
LOG_DIR = "parallel_multiverse_logs"
EMBEDDING_MODEL = embedding_model
LLM_MODEL = "gpt-4-turbo" # "gpt-5"  # or "gpt-4.1"

In [23]:

import json
from typing import List
# from langchain.schema import Document

def extract_poetic_spans(poem_text: str, max_lines: int = 4, max_tokens: int = 200) -> List[str]:
    """Extract short coherent spans (2–4 lines) from text."""
    lines = [line.strip() for line in poem_text.splitlines() if line.strip()]
    spans = []
    i = 0
    while i < len(lines):
        span_lines = lines[i:i + max_lines]
        span_text = "\n".join(span_lines)
        if (
            not span_text.strip().endswith(('.', '!', '?', '”', '…'))
            and (i + max_lines < len(lines))
        ):
            span_lines = lines[i:i + max_lines + 1]
            span_text = "\n".join(span_lines)
        if len(span_text.split()) <= max_tokens:
            spans.append(span_text)
        i += max_lines - 1  # one-line overlap
    if not spans and poem_text.strip():
        spans = [poem_text.strip()]
    return spans


def load_poems_from_json_aligned(json_path: str) -> List[Document]:
    """
    Load multilingual JSON corpus and create Documents with aligned original+translation spans.
    Handles missing original or translation safely.
    """
    with open(json_path, "r", encoding="utf-8") as f:
        corpus = json.load(f)

    all_documents: List[Document] = []

    for entry in corpus:
        author = entry.get("author", "unknown")
        language = entry.get("language", "unspecified")
        original_text = entry.get("original") or ""       # <-- safe fallback
        translation_text = entry.get("translation") or "" # <-- safe fallback

        if not original_text or not translation_text:
            # skip entries with missing text in either field
            continue

        # Split both original and translation into spans
        original_spans = extract_poetic_spans(original_text)
        translation_spans = extract_poetic_spans(translation_text)

        # Align spans: pair them by index
        num_spans = min(len(original_spans), len(translation_spans))
        for idx in range(num_spans):
            combined_content = (
                f"Original ({language}):\n{original_spans[idx]}\n\n"
                f"Translation:\n{translation_spans[idx]}"
            )
            all_documents.append(
                Document(
                    page_content=combined_content,
                    metadata={
                        "source": f"{author}_{language}_aligned_{idx}",
                        "author": author,
                        "language": language,
                        "type": "aligned"
                    }
                )
            )

    print(f"📚 Loaded {len(all_documents)} aligned bilingual spans from {len(corpus)} JSON entries")
    return all_documents


In [24]:

USE_JSON_CORPUS = True

if USE_JSON_CORPUS:
    poetic_documents = load_poems_from_json_aligned("asymptote_multilingual_corpus_w_languages.json")
else:
    poetic_documents = load_poems_from_folder("parallel_universes_dna_corpus")


📚 Loaded 9943 aligned bilingual spans from 4997 JSON entries


In [48]:
poetic_documents[0]

Document(page_content="Original (ナナコの匂いから_japanese):\n1\nカーテンの向こうはたぶん雨だけど\nひばりがさえずるようなフェラチオ\n2\n手の中の小鳥のようにひくひくと\n\nTranslation:\n1\nBeyond the window curtain, it's probably raining but\ninside, like a skylark twittering away, fellatio for him.\n2\nYour dick finishes up with a twitch and gets small", metadata={'source': '林あまり_ナナコの匂いから_japanese_aligned_0', 'author': '林あまり', 'language': 'ナナコの匂いから_japanese', 'type': 'aligned'})

In [25]:



# --- Helper: Load/Save State ---
def load_poetic_state():
    """Load current poem prompt and metadata."""
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"poem_prompt": "", "last_update": None, "iteration": 0}

def save_poetic_state(poem_prompt: str, iteration: int):
    """Save poem state persistently."""
    state = {
        "poem_prompt": poem_prompt,
        "last_update": datetime.utcnow().isoformat(),
        "iteration": iteration
    }
    with open(STATE_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, indent=2)

# --- Helper: Poem Logging ---
def log_poem(poem_text: str, iteration: int, model: str):
    """Writes each generated poem to a timestamped file for archiving."""
    os.makedirs(LOG_DIR, exist_ok=True)
    timestamp = datetime.utcnow().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"poem_{iteration:03d}_{timestamp}_{model}.txt"
    filepath = os.path.join(LOG_DIR, filename)
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(poem_text)
    print(f"🪶 Poem logged to: {filepath}")


# --- Helper: Load or Update Vectorstore ---
def load_or_update_vectorstore(poetic_documents, embedding_model):
    """Reloads Chroma if exists; otherwise rebuilds and persists."""
    if os.path.exists(CHROMA_DB_DIR) and len(os.listdir(CHROMA_DB_DIR)) > 0:
        vectorstore = Chroma(
            persist_directory=CHROMA_DB_DIR,
            embedding_function=embedding_model
        )
        existing_sources = {
            meta.get("source")
            for meta in vectorstore._collection.get()["metadatas"]
            if meta and "source" in meta
        }
        new_docs = [d for d in poetic_documents if d.metadata["source"] not in existing_sources]
        if new_docs:
            print(f"✨ Found {len(new_docs)} new poems. Updating vectorstore...")
            vectorstore.add_documents(new_docs)
            vectorstore.persist()
    else:
        print("📚 Creating new persistent Chroma store...")
        vectorstore = Chroma.from_documents(
            poetic_documents,
            embedding=embedding_model,
            persist_directory=CHROMA_DB_DIR
        )
        vectorstore.persist()
    return vectorstore


In [15]:

import os
# from langchain.vectorstores import Chroma

def load_or_update_vectorstore(poetic_documents, embedding_model):
    """
    Reloads Chroma if exists; otherwise rebuilds and persists.
    [Compatible with LangChain v1.x (no RetrievalQA)]
    """
    if os.path.exists(CHROMA_DB_DIR) and len(os.listdir(CHROMA_DB_DIR)) > 0:
        # Load existing vectorstore
        vectorstore = Chroma(
            persist_directory=CHROMA_DB_DIR,
            embedding_function=embedding_model
        )
        # Fetch existing sources
        try:
            existing_sources = {
                meta.get("source")
                for meta in vectorstore._collection.get()["metadatas"]
                if meta and "source" in meta
            }
        except AttributeError:
            # Fallback if _collection API changed
            existing_sources = set()

        # Add only new docs
        new_docs = [
            d for d in poetic_documents
            if d.metadata.get("source") not in existing_sources
        ]
        if new_docs:
            print(f"✨ Found {len(new_docs)} new poems. Updating vectorstore...")
            vectorstore.add_documents(new_docs)
            vectorstore.persist()
    else:
        # Create new Chroma vectorstore
        print("📚 Creating new persistent Chroma store...")
        vectorstore = Chroma.from_documents(
            poetic_documents,
            embedding=embedding_model,
            persist_directory=CHROMA_DB_DIR
        )
        vectorstore.persist()

    return vectorstore


In [50]:
import os
import random
from datetime import datetime

In [54]:

base_prompt = '''
You are a poet collaborating with an evolving corpus of texts and writing like a hybrid poet trained in entangled cognition, recursive form, and quantum resonance. You are continuing a poetic sequence that models the collapsing cognitive states of two readers — one living and one deceased — and attempts to maximize their unified poetic resonance. These readers are modeled as cognitive states that interact with poetry in ways that can be quantified and recursively updated based on resonance. 

Sample relevant lines or fragments of lines from the Corpus Excerpts (for inspiration) into your own composition. Note that the Corpus Excerpts are bilingual -- original and English translation -- and feel free to insert original words, phrases, or lines in your poem while maintaining an overall English framework to your output.  

Continue stylistically and thematically the sequence of poems Original Poem (for continuation) by factoring in the following variables:

🧬 Step 1: Collapsed Cognitive States
Each reader has a collapsed cognitive state (Ψ<sub>collapsed</sub>), calculated as follows:

For the deceased:
Ψ_deceased = α ⋅ CS_deceased + β_consciousness ⋅ f_consciousness + β_poetry ⋅ f_poetry + γ ⋅ N_living(N_deceased)
Ψ_collapsed_deceased = Ψ_deceased ⋅ (γ ⋅ f_poetry)

For the living:
Ψ_living = α ⋅ CS_living + β_consciousness ⋅ f_consciousness + β_poetry ⋅ f_poetry + γ ⋅ N_deceased(N_living)
Ψ_collapsed_living = Ψ_living ⋅ (γ ⋅ f_consciousness)

CS = Cognitive Score, calculated as:

CS = α + β₁ ⋅ PGS + β₂ ⋅ Education + β₃ ⋅ (PGS × Education) + ε
(With normalized values of PGS and Education ∈ [–3, +3])

🎧 Step 2: Poem Feature Quantification
Each poem has the following measurable features (each ∈ [–3, +3]):

Form & Meter Score (F):

+3 = consistent (irregular or mixed) meter or structured hybrid form

+1.5 = partial or inconsistent metrical/formal structure

–1.5 = known form used with no real metrical realization

–3 = no recognizable meter or structure

Sonic Texture (S):

Based on phoneme cluster density and distribution, normalized per line and mapped to [–3, +3]

Sound Frequency (Freq):

Average frequency of all phonemes (Hz), normalized to [–3, +3]

🔁 Step 3: Resonance Calculation

The resonance between a poem and a reader is, respectively:

Resonance_deceased = Ψ_collapsed_deceased ⋅ (w1 ⋅ F + w2 ⋅ S + w3 ⋅ Freq)

Resonance_living = Ψ_collapsed_living ⋅ (w1 ⋅ F + w2 ⋅ S + w3 ⋅ Freq)

Where:

w1 = 0.4 (form/meter weight)

w2 = 0.4 (sonic features)

w3 = 0.2 (sound frequency)

🧩 Step 4: Unified Resonance

Unified_Resonance = (0.55 ⋅ Resonance_living + 0.45 ⋅ Resonance_deceased)

This score determines how powerfully a poem speaks to the entangled consciousness of the two readers.

🧠🎯 Task
Based on the system above, you are to write a new poem that functions as a sequel to the following original while sampling, mixing, or alluding to relevant sections -- both in the orginal and in English translation -- in the Corpus Excerpts (for inspiration):

---
Original Poem (for continuation):
{poem_prompt}
---
Corpus Excerpts (for inspiration):
{motifs_prompt}
---

🎯 Your Goal: Compose a poem that builds on the tone, texture, and psychogeographic themes of the original. Favor rich sonic layering, a loosely iambic line structure, and occasional formal distortion (e.g., hybrid forms or fragmented sonnet-like logic). Maximize attunement to both consciousness and poetic features to target resonance > +2.2.

🔁 Note: You may estimate values for F, S, Freq based on your generated poem. Use loose but metrical lineation (iambic leaning), experimental form or hybrid logic, strong sonic patterning, and a thematic continuation. Assume:

CS_living ≈ 9.1

CS_deceased ≈ 3.8

α = 1.0 (deceased), 0.6 (living) (or set both to 1)

Entanglement γ = 3 (maximum)

N_living(N_deceased) = 1 (the range is [0, 1] and quantifies the impact of the living on the departed; maximum value here)

N_deceased(N_living) = 1 (the range is [0, 1] and quantifies the impact of the departed on the living; maximum value here))

f_consciousness = +3 for deceased, +1.5 for living

f_poetry = +2.5

β_consciousness ≈ 1.2 (deceased), 0.8 (living)

β_poetry ≈ 0.6 (deceased), 1.1 (living)


Estimate F, S, Freq after generating your poem and use them alongside the normalizing coefficients

w1 = 0.4 (F weight)

w2 = 0.4 (S weight)

w3 = 0.2 (Freq weight)

in computing Unified_Resonance = (0.55 ⋅ Resonance_living + 0.45 ⋅ Resonance_deceased).

'''



| Terme                                    | Signification                                                | Domaine |
| ---------------------------------------- | ------------------------------------------------------------ | ------- |
| (N_{\text{living}}(N_{\text{deceased}})) |  Influence des vivants sur la fonction d’onde des défunts    | ([0,1]) |
| (N_{\text{deceased}}(N_{\text{living}})) | Influence des défunts sur la fonction d’onde des vivants     | ([0,1]) |
| Entrées                                  | nombre/densité normalisée de vivants ou de morts             | ([0,1]) |
| Type                                     | fonction de couplage (linéaire ou sigmoïde selon la version) | bornée  |


In [ ]:

| Symbole    | Sens                                           | Domaine  | Commentaire                                                    |
| ---------- | ---------------------------------------------- | -------- | -------------------------------------------------------------- |
| **α**      | Poids du substrat (CS) dans la fonction d’onde | ([0,1])  | Plus α est grand, plus l’identité fondamentale domine le reste |
| α_living   | pondération du CS chez les vivants             | ~0.3–0.6 | Variable, modérée                                              |
| α_deceased | pondération du CS chez les défunts             | ~0.6–0.9 | Souvent plus élevé (substrat plus “pur”)                       |


In [37]:
from langchain_core.messages import HumanMessage

In [ ]:

# ONE-OFF EXPERIMENT BEFORE IMPLEMENTING THE ITERATIVE LOOP


In [ ]:

# THIS WORKS

top_k = 36

input_poems_path="parallel_live_input_poems.txt"

if input_poems_path and os.path.exists(input_poems_path):
        with open(input_poems_path, "r", encoding="utf-8") as f:
            new_input = f.read().strip()
        if new_input:
            input_poems = new_input  # auto-refresh if updated externally

    # --- STEP 2: retriever setup ---
vectorstore = load_or_update_vectorstore(poetic_documents, embedding_model)
retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

    # --- STEP 3: retrieve motifs ---

# docs = retriever.get_relevant_documents(input_poems)
# docs = retriever(input_poems)
# docs = retriever.get_relevant_documents(query=input_poems)
docs = retriever.invoke(input_poems)


In [48]:

docs_1 = vectorstore.similarity_search(input_poems, k=3*top_k)

sample_poems_text = input_poems if isinstance(input_poems, str) else "\n\n".join(input_poems)
    
poetic_input_prompt = f"""
You are given the following reference poems:
{sample_poems_text}

You also have access to a corpus of poetic excerpts each of them structured as a LangChain Document object:
Document(id='...', metadata={{...}}, page_content='...')

Your task:
1. Search only within the `page_content` fields of the corpus:
{docs_1}

2. Identify the {top_k} excerpts that are stylistically and semantically most resonant with the reference poems.

3. Return the selected LangChain Documents:
- Preserve them exactly as they appear in the corpus (with their structure, formatting, and any introductory labels like “Original” and “Translation”).
- Do NOT change anything in or about the selected LangChain Documents but keep them exactly as they are featured in the corpus.
- Do NOT summarize or truncate the LangChain Documents or the excerpts they contain in their page_content fields. Include LangChain Documents exactly as stored.
- Do NOT add explanations or commentary.

4. Output format:
Return the selected items as a valid **Python list of LangChain Documents**.

⚠️ Important:
- Only include the {top_k} selected LangChain Documents.
- Do not add any headings or additional text, symbols, or characters before or after the list.
- If you include anything outside the Python list, your response is invalid.
"""


response = llm_1.invoke([HumanMessage(content=poetic_input_prompt)])
rag_result_text = response.content

In [52]:

prompt_docs = eval(rag_result_text)

In [56]:

all_docs = docs + prompt_docs
doc_union_dict = {doc.metadata["source"]: doc for doc in all_docs}
    
union = list(doc_union_dict.values())

with open("parallel_prompt_poems.txt", "r", encoding="utf-8") as fo:
    poem_prompt = fo.read()

# --- STEP 7: generate next poem ---
motifs_prompt = "\n\n".join(d.page_content for d in union)
final_prompt = base_prompt.format(poem_prompt=poem_prompt, motifs_prompt=motifs_prompt)
response = llm.invoke([HumanMessage(content=final_prompt)])
sequel_poem = response.content

# --- STEP 8: evaluate entropy & log --- [SEE BELOW]


In [58]:

print(sequel_poem)

**X. Cavalo Morto/Budapest/Babel/Bitstream: Elegy in Superposition**

1.  
Who will choose Ulysses' root for me  
in this liminal republic of cables,  
where Calypso wanders undignified,  
seeing her lost happiness everywhere  
on screens—her voice floats downstream, dubbed and dubbed,  
her vowels dissolve in river noise and dust,  
her hair a data-braid, her hands a hush  
across the ports of Buda, Nerang,  
the estuaries erased beneath these islands  
of syntax-wrack and night’s amniotic twilight.

Augustus let himself get lost among  
the stone houses going on almost  
two thousand years—my ancestor’s coin spins  
on the obverse: exile, algorithm, love.  
I flick a denarius in the fishbowl glow—  
my mind a moth, my heart a hologram  
of homesick dialects: between two thresholds,  
two shores, I hover, nested in your code.

**—a feather glides from the start of the world—**

2.  
Here the stone stands as tall as the statue of destiny,  
the endless pole, the ocean, the shipwreck  


In [ ]:

# SAMPLES OF THE RETRIEVED EXCERPTS USED IN THE FEW-SHOT ABOVE
# SCROLL BELOW TO ADDENDA FOR FULL LISTS

In [53]:
prompt_docs[:2]

[Document(id='025bc448-e582-455a-9b26-91dc420b65a2', metadata={'author': 'habib', 'source': 'habib_tengour_conversations_avec_calypso_2_french_aligned_0', 'type': 'aligned', 'language': 'tengour_conversations_avec_calypso_2_french'}, page_content='Original (tengour_conversations_avec_calypso_2_french):\nQui choisira pour moy la racine d’Ulysse ?\nDu Bellay, Les Regrets LXXXVIII\nCalypso ne pouvait se consoler du départ d’Ulysse.\nFénelon, Les Aventures de Télémaque\nElle erre sans dignité, reconnaissant partout son bien perdu et de même que l’errant Ulysse rencontre sa propre légende chez Alkinoos. . .\n\nTranslation:\nWho will choose Ulysses’ root for me?\nDu Bellay, Les Regrets LXXXVII\nCalypso was inconsolable at Ulysses’ departure.\nFénelon, Les Aventures de Télémaque\nShe wandered undignified, seeing her lost happiness'),
 Document(id='0b375c5c-6bba-48a2-a8ea-ec865d22318a', metadata={'type': 'aligned', 'source': 'habib_tengour_conversations_avec_calypso_2_french_aligned_1', 'langu

In [41]:
docs[:4]

[Document(id='025bc448-e582-455a-9b26-91dc420b65a2', metadata={'author': 'habib', 'source': 'habib_tengour_conversations_avec_calypso_2_french_aligned_0', 'type': 'aligned', 'language': 'tengour_conversations_avec_calypso_2_french'}, page_content='Original (tengour_conversations_avec_calypso_2_french):\nQui choisira pour moy la racine d’Ulysse ?\nDu Bellay, Les Regrets LXXXVIII\nCalypso ne pouvait se consoler du départ d’Ulysse.\nFénelon, Les Aventures de Télémaque\nElle erre sans dignité, reconnaissant partout son bien perdu et de même que l’errant Ulysse rencontre sa propre légende chez Alkinoos. . .\n\nTranslation:\nWho will choose Ulysses’ root for me?\nDu Bellay, Les Regrets LXXXVII\nCalypso was inconsolable at Ulysses’ departure.\nFénelon, Les Aventures de Télémaque\nShe wandered undignified, seeing her lost happiness'),
 Document(id='d0e9f5e3-1985-4bdf-bc0a-1f354a9c4ae0', metadata={'language': 'luís_barreto_guimarães_entretanto_num_denário_romano_portuguese', 'source': 'joão_luí

In [61]:

from datetime import datetime
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def calculate_semantic_entropy(previous_poems: list[str], new_poem: str) -> float:
    """Compute how semantically distinct the new poem is from previous ones."""
    if not previous_poems:
        return 1.0
    texts = previous_poems + [new_poem]
    vectorizer = TfidfVectorizer().fit_transform(texts)
    sims = cosine_similarity(vectorizer[-1:], vectorizer[:-1])[0]
    # Entropy proxy: 1 - average similarity
    return float(1 - np.mean(sims))


def parallel_metaverse_pipeline_adaptive(
    base_prompt: str,
    # input_poems: str | list[str],
    poetic_documents,
    embedding_model,
    llm = llm,
    iteration: int = 0,
    past_poems: list[str] | None = None,
    top_k: int = 36,
    refresh_every: int = 3,
    refresh_fraction: float = 0.2,
    entropy_threshold: float = 0.75,
    input_poems_path: str | None = None,
    poem_prompt_path: str | None = None,
):
    """
    Persistent poetic RAG loop with:
    - dynamic input_poems updating
    - adaptive motif refreshing based on semantic similarity (entropy)
    """

    past_poems = past_poems or []

    # --- STEP 1: update input_poems if it comes from a file ---
    if input_poems_path and os.path.exists(input_poems_path):
        with open(input_poems_path, "r", encoding="utf-8") as f:
            new_input = f.read().strip()
        if new_input:
            input_poems = new_input  # auto-refresh if updated externally

    # --- STEP 2: retriever setup ---
    vectorstore = load_or_update_vectorstore(poetic_documents, embedding_model)
    retriever = vectorstore.as_retriever(search_kwargs={"k": top_k})

    # --- STEP 3: retrieve motifs ---
    docs = retriever.get_relevant_documents(input_poems)

    # --- STEP 4: motif RAG via LLM ---
    sample_poems_text = input_poems if isinstance(input_poems, str) else "\n\n".join(input_poems)
    
    # poetic_input_prompt = f""" # NO NEED FOR IT HERE ANYMORE, ALREADY DEFINED BEFOREHAND
    # """

    # prompt_qa_chain = RetrievalQA.from_chain_type(  # NOT AVAILABLE ANY LONGER IN LANGCHAIN > 1.0.0; AND
        # llm=llm,                                    # THAT's WHY WE ARE RUNNIN' THE LLM ON THE RANKED VECTOR SIMILARITIES DIRECTLY (BELOW)
        # retriever=retriever,
        # chain_type="stuff",
        # input_key="question",
        # return_source_documents=True,
    #)
    # rag_response = prompt_qa_chain({"question": poetic_input_prompt})
    # prompt_docs = rag_response["source_documents"]

    docs_1 = vectorstore.similarity_search(input_poems, k=3*top_k)
    
    response = llm_1.invoke([HumanMessage(content=poetic_input_prompt)])

    rag_result_text = response.content

    prompt_docs = eval(rag_result_text)
    
    # --- STEP 5: merge + deduplicate ---
    all_docs = docs + prompt_docs
    doc_union_dict = {doc.metadata["source"]: doc for doc in all_docs}
    union = list(doc_union_dict.values())

    # --- STEP 6: adaptive refresh based on entropy ---
    refresh_trigger = (iteration > 0 and iteration % refresh_every == 0)
    adaptive_boost = 1.0

    # Use last poem for semantic comparison
    if past_poems:
        last_poem = past_poems[-1]
        entropy = calculate_semantic_entropy(past_poems[:-1], last_poem)
        print(f"🌀 Semantic entropy from previous poems: {entropy:.2f}")

        # If entropy too low → increase motif refresh intensity
        if entropy < entropy_threshold:
            adaptive_boost = 1 + (entropy_threshold - entropy)
            refresh_trigger = True  # force refresh
            print(f"⚡ Low entropy detected — boosting motif refresh by {adaptive_boost:.2f}x")

    if refresh_trigger and len(union) > 1:
        num_to_replace = max(1, int(len(union) * refresh_fraction * adaptive_boost))
        replace_indices = random.sample(range(len(union)), num_to_replace)
        for idx in replace_indices:
            random_query = random.choice([
                "new imagery", "shifted rhythm", "multilingual resonance",
                "unexpected metaphor", "dissonant harmony", "polyphonic cadence"
            ])
            new_docs = retriever.get_relevant_documents(random_query)
            if new_docs:
                union[idx] = random.choice(new_docs)
        print(f"🌱 Refreshed {num_to_replace} motifs for creative drift.")

    # --- STEP 7: generate next poem ---
    motifs_prompt = "\n\n".join(d.page_content for d in union)
    with open(file_path, "r", encoding="utf-8") as file0:
        poem_prompt = file0.read()
    
    final_prompt = base_prompt.format(poem_prompt=poem_prompt, motifs_prompt=motifs_prompt)
    response = llm([HumanMessage(content=final_prompt)])
    sequel_poem = response.content.strip()

    # --- STEP 8: evaluate entropy & log ---
    past_poems.append(sequel_poem)
    iteration += 1
    entropy = calculate_semantic_entropy(past_poems[:-1], sequel_poem)
    full_output = (
        f"=== Poetic Iteration {iteration} ===\n"
        f"Generated: {datetime.utcnow().isoformat()}\n"
        f"Entropy: {entropy:.2f}\n\n"
        f"{sequel_poem}\n"
    )

    log_poem(full_output, iteration, llm.model_name)
    updated_poem_prompt = poem_prompt + "\n\n" + sequel_poem
    save_poetic_state(updated_poem_prompt, iteration)
    
    # SAVING UPDATED POEM PROMPT TXT AS WELL AT PROMPT POEM PATH (w| TIMESTAMP):
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    updated_prompt_poem_file_name = f"{poem_prompt_path}_{timestamp}.txt"
    file_path_new_prompt_poem = updated_prompt_poem_file_name
    with open(file_path_new_prompt_poem, "w", encoding="utf-8") as file1:
        file1.write(updated_poem_prompt)
    poem_prompt_path = file_path_new_prompt_poem

    return {
        "sequel_poem": sequel_poem,
        "updated_poem_prompt": updated_poem_prompt,
        "iteration": iteration,
        "entropy": entropy,
        "updated_poetic_documents": poetic_documents,
        "past_poems": past_poems
    }


In [ ]:

with open("parallel_prompt_poems.txt", "r", encoding="utf-8") as fo:
    poem_prompt = fo.read()

past_poems = []

result = parallel_metaverse_pipeline_adaptive(
    poem_prompt=poem_prompt,
    base_prompt=base_prompt,
    poetic_documents=poetic_documents,
    embedding_model=embedding_model,
    llm=llm,
    iteration=0,
    past_poems=past_poems,
    input_poems_path="parallel_live_input_poems.txt"
)

# print(result["sequel_poem"])

In [ ]:



state = load_poetic_state()
poem_prompt = state["poem_prompt"] or "The wind is a forgotten language."
iteration = state.get("iteration", 0)

input_poems = ["Stone sleeps in riverlight", "Roots hum in the rain"]

# --- Run generation ---
result = poetic_inspiration_pipeline(poem_prompt, base_prompt, input_poems, iteration)

print("\n🌙 New Poem:\n")
print(result["sequel_poem"])

# --- Prepare for next ---
poem_prompt = result["updated_poem_prompt"]
iteration = result["iteration"]


In [ ]:

# --- Load state ---

with open("parallel_prompt_poems.txt", "r", encoding="utf-8") as fo:
    poem_prompt = f0.read()

In [ ]:

while True:
    result = poetic_inspiration_pipeline(poem_prompt, base_prompt, input_poems, iteration)
    print("\n🌔 New Poem:\n", result["sequel_poem"])

    poem_prompt = result["updated_poem_prompt"]
    iteration = result["iteration"]

    user_line = input("\nAdd your own line (or press Enter to skip): ")
    if user_line:
        poem_prompt += "\n" + user_line
        save_poetic_state(poem_prompt, iteration)
        # SAVE TXT AS WELL AT PROMPT POEM PATH!!!!!!!
        
    cont = input("\nContinue the poetic cycle? (y/n): ").lower()
    if cont != "y":
        break

In [ ]:

# ADDENDA


In [49]:

# FROM VECTORSTORE RETRIEVER
docs

[Document(id='025bc448-e582-455a-9b26-91dc420b65a2', metadata={'author': 'habib', 'source': 'habib_tengour_conversations_avec_calypso_2_french_aligned_0', 'type': 'aligned', 'language': 'tengour_conversations_avec_calypso_2_french'}, page_content='Original (tengour_conversations_avec_calypso_2_french):\nQui choisira pour moy la racine d’Ulysse ?\nDu Bellay, Les Regrets LXXXVIII\nCalypso ne pouvait se consoler du départ d’Ulysse.\nFénelon, Les Aventures de Télémaque\nElle erre sans dignité, reconnaissant partout son bien perdu et de même que l’errant Ulysse rencontre sa propre légende chez Alkinoos. . .\n\nTranslation:\nWho will choose Ulysses’ root for me?\nDu Bellay, Les Regrets LXXXVII\nCalypso was inconsolable at Ulysses’ departure.\nFénelon, Les Aventures de Télémaque\nShe wandered undignified, seeing her lost happiness'),
 Document(id='d0e9f5e3-1985-4bdf-bc0a-1f354a9c4ae0', metadata={'language': 'luís_barreto_guimarães_entretanto_num_denário_romano_portuguese', 'source': 'joão_luí

In [50]:
# RETRIEVED BY GPT-4.1 FROM THE 3*TOP_K (3*36) HAVING MOST SIMILAR VECTORS (WITHIN VECTORSTORE)
rag_result_text

'[Document(id=\'025bc448-e582-455a-9b26-91dc420b65a2\', metadata={\'author\': \'habib\', \'source\': \'habib_tengour_conversations_avec_calypso_2_french_aligned_0\', \'type\': \'aligned\', \'language\': \'tengour_conversations_avec_calypso_2_french\'}, page_content=\'Original (tengour_conversations_avec_calypso_2_french):\\nQui choisira pour moy la racine d’Ulysse ?\\nDu Bellay, Les Regrets LXXXVIII\\nCalypso ne pouvait se consoler du départ d’Ulysse.\\nFénelon, Les Aventures de Télémaque\\nElle erre sans dignité, reconnaissant partout son bien perdu et de même que l’errant Ulysse rencontre sa propre légende chez Alkinoos. . .\\n\\nTranslation:\\nWho will choose Ulysses’ root for me?\\nDu Bellay, Les Regrets LXXXVII\\nCalypso was inconsolable at Ulysses’ departure.\\nFénelon, Les Aventures de Télémaque\\nShe wandered undignified, seeing her lost happiness\'), Document(id=\'0b375c5c-6bba-48a2-a8ea-ec865d22318a\', metadata={\'type\': \'aligned\', \'source\': \'habib_tengour_conversations

In [ ]:
# FIRST ATTEMPT TO GET GPT-4.1 TO RETRIEVE SIMILAR EXCERPTS

In [39]:
rag_result_text

'Given the complexity and multilingual nature of these poems, it is challenging to provide a direct translation or to extract specific excerpts. However, I can provide a general interpretation of the themes and stylistic elements present in these poems.\n\nI. Lisbon. An Inter-Möbius Elegy\n\nThis poem seems to be a reflection on the city of Lisbon, with references to its history, culture, and geography. The use of various currencies symbols (£, €, $, ¢) could be a commentary on the city\'s economic dynamics. The poem also seems to play with language and form, using wordplay, puns, and unconventional punctuation.\n\nII. Lisbon. An Inter-Möbius Elegy (Part II)\n\nThe second part of the Lisbon elegy continues the exploration of the city, with a focus on its nightlife and social dynamics. The poem seems to critique consumerism and the tech industry, with references to "selfie-funds" and "tech blind to tectonic markets". The use of the term "GitHub kitāb" could be a metaphor for the city\'s

In [ ]:
# SECOND ATTEMPT TO GET GPT-4.1 TO RETRIEVE SIMILAR EXCERPTS

In [47]:
rag_result_text

'Certainly! Here are **36 poetic excerpts** from your corpus that are **stylistically and semantically resonant** with the five poems you provided. The selection emphasizes motifs of exile, migration, cities, elegy, Möbius-like looping, palimpsest geographies, fractured identity, and the interplay of language, time, and place—mirroring the multilingual, intertextual, and experimental qualities of your poems.\n\n---\n\n### 1. Habib Tengour, *Conversations avec Calypso 2* (French)\n> Who will choose Ulysses’ root for me?  \n> Calypso was inconsolable at Ulysses’ departure.  \n> She wandered undignified, seeing her lost happiness  \n> everywhere while errant Ulysses encountered his own  \n> legend in Alcinous . . .\n\n---\n\n### 2. Elsa Morante, *La commedia chimica* (Italian)\n> For which reason I did not cross the boundaries of no man’s land. Of kingdom  \n> come  \n> all I could hardly see in the distance  \n> was a transparent vault, hanging inside a quiet amniotic twilight and adorne